# Getting Anilist anime-lists
* See GetUserAnimeLists.ipynb for more info

In [ ]:
name = "anilist"

In [ ]:
%run Anilist.ipynb

## Get usernames

In [ ]:
def read_usernames():
    users = list(range(1, 1000000)) + list(range(5100000, 6100000))
    np.random.shuffle(users)
    return users

In [ ]:
%run GetUserAnimeLists.ipynb